(Analysis)=
# Analyzing Simulation Results

Once you have [setup your transformation pathway](The_Cycle), defined necessary [intermediate states](intermediate_states), [run your equilibrium simulation](Runnning_Sims), and acquired the [correct data from the runs](Pulling_Sim_Data), you may now analyze the data to get free energies. Each of the free energy techniques discussed in the theory section of these fundamentals talks about how you get the free energies, but will also be recapped here. This page also talks about how you can get uncertainty estimates with the [bootstrap method](#bootstrap).

## Calculating Free Energies
Shown below is a brief summary of the information needed and the method to calculating free energy with the various techniques. If you have read the pages for the free energy techniques, this will be a recap.

* [EXP](exp_avg) is a very straightforward calculation. You will need the $\Delta U$ from a given pair of states. From there, you can calculate the free energy from the effective sample size found by [correcting for correlation/subsampling](correlation). Variances will be additive.
* [TI](TI) needs $\frac{dU}{d\lambda}$ and the average at each of $K$ states needs calculated. Since its more common to have discrete $\lambda$ states, you will need to choose an appropriate [weighting method](TI_FE) to correctly calculate the free energy from the [uncorrelated/subsampled](correlation) data. By-state Variance will not be additive, but this is [simple to account for](TI_error).
* [WHAM](WHAM) requires binning all your results then calculating $\Delta U$ from all states. It is highly recommended that you take advantage of the tools already out there for calculating WHAM instead of writing your own. There is no direct way to calculate variance, and so methods such as [bootstrap sampling](#bootstrap) are needed.
* [BAR](BAR) requires the $\Delta U$ from two states. An iterative, often numeric, solution is needed to find the free energy, but this is relatively straightforward and can be done for each pair of states. The variance does not have as simple a relationship as with TI, and methods such as [bootstrap sampling](#bootstrap) are recommended. There is a Python implementation of BAR available with [PyMBAR](https://github.com/choderalab/pymbar).
* [MBAR](MBAR) also requires $\Delta U$, but it needs it for all combination of states. Again, an iterative set of solutions is needed and it is not recommended that users attempt to code MBAR themselves. Instead, please take advantage of a Python implementation available called [PyMBAR](https://github.com/choderalab/pymbar). Error estimates are directly available with MBAR, however, it can still benefit from methods such as [bootstrap sampling](#bootstrap).

(bootstrap)=
## Bootstrap Sampling

Bootstrap sampling is a statistical tool in when we can get decent estimates for the uncertainty with very limited data;{cite}`Efron1993` this method works with all techniques presented in the [fundamentals section](fundamentals). We carry out bootstrap sampling as the time taken to do bootstrap is often substantially less than the time it would take to generate new samples.

If we assume we have some function, $F(x)$ that is computed from $N$ data samples, $x_1,x_2\ldots,x_N$, and we have that $\lim_{N\to\infty} F(x) = \overline{F}$ where $\overline{F}$ is a constant. As an example, say that $F(x)$ is the simple average of all $x$ 

$\displaystyle F(x)=\frac{1}{N}\sum\limits_{i=1}^N x_i$,

or the average of some function $ X(x)$ such that

$\displaystyle F(x)=\frac{1}{N}\sum\limits_{i=1}^N X(x_i)$;

we are not limited to such simple choices as the functions could be as complicated the MBAR or WHAM free energy estimators. To calculate the bootstrap variance, carry out the following procedure.

1. **Pick with replacement** $n$ samples from the complete list of samples $\vec{x}\{x_1,\ldots,x_N\}$ to create a new set of samples $\vec{x}_i$. Since you are picking randomly with replacement, there are bound to be repeated samples. This method is called sampling from the *empirical distribution,* that is, sampling from the distribution we measured, rather than true distribution.  For example, if $\vec{x} = \{1,2,6,4,3\}$, a possible set of $\vec{x}_i$ might be $\vec{x}_i = \{6,1,1,4,4\}$; note that $\vec{x}_i$ is *the same size* as the original $\vec{x}$ set.
1. **Compute $\hat{F}_i = F(x_i)$.** That is, find the estimate for your function based on the bootstrap sample taken from the empirical distribution. If we were finding simple averages, we would calculate the average of the bootstrap sample. If we were calculating free energies with MBAR, we would generate a bootstrap sample at *each* $K$ state and estimate the full free energy at each state based on the bootstrap samples.
1. **Repeat steps 1 and 2** for $M$ number of times. You will need at least 50-200 times to obtain accurate variances {cite}`Efron1993`. If the calculation in step 2 is cheap, then $M$ can be very large to get even better estimates of the variance; relative uncertainty scales as $M^{-1/2}$ and can take more than 1000 steps to get variance within 1% convergence.
1. **Compute the variance** $\mathrm{var}\left(\hat{F}\right)$, which is the variance over the $M$ bootstrap values. The standard deviation is then just the square root of the bootstrap variance.

The values of $\hat{F_i}$ are the bootstrap distribution of the function $F(x)$. In the large sample limit, the bootstrap distribution will converge to the true distribution of $\overline{F}$ under most conditions, but will still be a good estimate for moderate values of $N$. If N is too low though, the bootstrap distribution will not have good agreement with the true distribution.

Bootstrap sampling can be used with *any* statistical estimator, regardless of how complicated. There is the extra overhead of calculating $F(x)$ $M$ times, but this is often negligible compared to how long it would take to collect $M$ times the number of data. For MBAR, the time required for bootstrap is only 5-10min, where as TI is only seconds. There is also a variant of bootstrap called "moving block bootstrapping" which accounts for [correlation time](correlation) *without* subsampling by taking random block of length $\tau$.{cite}`Efron1993`

### Example of Bootstrap Method
```{figure} ../images/Bootstrap_Comparison.png
---
width: 400px
name: bootstrapExample
---
A comparison of the estimated distribution of exponential average (EXP) results using bootstrap sampling to the true distribution obtained using multiple draws. Each draw consists of N samples from a normal distribution with mean 0 and variance 1. For N = 10 samples, bootstrap sampling does not give the true distribution; for N = 1000 samples, bootstrap sampling yields a very close match to the distribution, with standard error 0:0437 vs 0:0414. Either 100,000 bootstrap samples or 100,000 independent draws were performed in each case.
```

Let's take a simple example where we will sample values of $x$ from a 1D Gaussian with mean 0 and variance 1 ($x$~$N(0,1)$). We will now compute [EXP](exp_avg) with $k_BT=1$ for this sampling by

$F(x_i) = -\ln \left[N^{-1}\sum_{i=1}^N \exp(-x_i)\right]$.

We then compute the distribution of free energies obtained by $N$ points for 100,000 bootstrap samples, and 100,000 independent samples (see figure on right). For small $N$, the distributions are not close, but nearly converge for larger $N$. Even at N=200, the variances are near identical. To emphasize the point, the "Repeated Samples" distribution uses data from $1000\times 100,000$ points collected, where the "Bootstrap Samples" needs only *one* set of 1000 samples run through the bootstrap process 100,000 times. The same amount of analysis is carried out in both cases, but the bootstrap samples only require 0.001% of the data collection. One catch to the bootstrap method that should be noted is that rare events requiring more than 1000 samples would not have been observed, so it still takes careful analysis to properly use bootstrap sampling.